<a href="https://colab.research.google.com/github/cornflake15/twitter_sentiment_analysis/blob/update/tweepy-vader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Twitter Sentiment Analysis

In [1]:
import tweepy
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [2]:
CONSUMER_API_KEY = '8fxjhgCnLZie0AjbLuntKT0EM'
CONSUMER_API_SECRET = 'd9oab3yrWQybhC64hk5YIXybhuIql20sUWvte2Z8vVQvKtM3Id'
ACCESS_TOKEN ='822833106-VKB22OHa8dXm2Hd5sEl5d5gWLmSfaH5EEmjA88DX'
ACCESS_TOKEN_SECRET = '22iR0cQfOvSQ77UFKKf4CHNLPvvNYEqzNUkugvubrLMx5'
auth = tweepy.OAuthHandler(CONSUMER_API_KEY, CONSUMER_API_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

### Utility Function

In [3]:
# Function to get trending topic by country name
def get_trending_topic_by_country(country, num_trends):
    api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
    trends_country = api.trends_available()
    for trend in trends_country:
        if trend['name'] == country:
            country_info = (trend['name'], trend['woeid'])
            print('Country: {}, WOEID: {}'.format(trend['name'], trend['woeid']))
    
    index = 1
    trending_topics = []
    trends_result = api.trends_place(country_info[1])
    for tren in trends_result[0]['trends'][:num_trends]: 
        print('#{}: {}'.format(index, tren['name']))
        trending_topics.append(tren['name'])
        index += 1
        
    return trending_topics

# Function to get tweets by keyword
def get_tweets_by_keyword(keywords, num_of_tweets, language, filename='', export_to_csv=False):
    """
    Use this function if you want export the tweet to CSV file
    keywords: tweet keyword
    num_of_tweets: number of tweets that your want to fetch
    language: tweet language
    filename: filename
    """
    api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
    tweets = tweepy.Cursor(api.search, q=keywords, lang=language).items(num_of_tweets)
    
    # These are all I can get with free Twitter API, you can get more with the premium one
    dt_tweets = [[tweet.user.name, tweet.user.screen_name, tweet.user.location, 
                  tweet.user.verified, tweet.user.followers_count,
                  tweet.user.friends_count, tweet.user.listed_count, 
                  tweet.created_at, tweet.text, tweet.coordinates, 
                  tweet.place, tweet.favorite_count] for tweet in tweets]
    
    df_tweets = pd.DataFrame(data=dt_tweets)
    df_tweets.columns = ['username', 'screen_name', 'user_location', 'user_verified', 
                     'followers_count', 'friends_count', 'listed_count', 'tweet_created_at',
                     'tweet_text', 'tweet_coordinates', 'tweet_place', 'tweet_favorite_count']
    
    
    if export_to_csv == True:
        export_to_csv(df_tweets, filename)
        
    return df_tweets

# Overiding function 
def get_tweets_by_keyword(keywords, num_of_tweets, language):
    """
    Use this function and its parameter if you don't want export to CSV
    """
    api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
    tweets = tweepy.Cursor(api.search, q=keywords, lang=language).items(num_of_tweets)
    
    # These are all I can get with free Twitter API, you can get more with the premium one
    dt_tweets = [[tweet.user.name, tweet.user.screen_name, tweet.user.location, 
                  tweet.user.verified, tweet.user.followers_count,
                  tweet.user.friends_count, tweet.user.listed_count, 
                  tweet.created_at, tweet.text, tweet.coordinates, 
                  tweet.place, tweet.favorite_count] for tweet in tweets]
    
    df_tweets = pd.DataFrame(data=dt_tweets)
    df_tweets.columns = ['username', 'screen_name', 'user_location', 'user_verified', 
                     'followers_count', 'friends_count', 'listed_count', 'tweet_created_at',
                     'tweet_text', 'tweet_coordinates', 'tweet_place', 'tweet_favorite_count']
        
    return df_tweets

# Function to remove unwanted string in the tweet
def remove_unwanted_string(dataframe):
    new_record = []
    tweet_text = dataframe.apply(lambda x: str(x).split())
    for record in tweet_text:
        for text in record:
            if 'RT' in text or '@' in text or '#' in text:
                record.remove(text)
        
        new_record.append(' '.join(record))
    dataframe = new_record
    return dataframe
            
# Function to print sentiments of the sentence
def sentiment_scores(sentence):
    # Create a SentimentIntensityAnalyzer object
    sid_obj = SentimentIntensityAnalyzer()
    
    # polarity_scores method of SentimentIntensityAnalyzer
    # object gives a sentiment dictionary
    # which contains pos, neg, neu, and compoind scores
    sentiment_dict = sid_obj.polarity_scores(sentence)
    
    return sentiment_dict
    
# Function export to CSV
def export_to_csv(dataframe, filename):
    print('\nExported to CSV')
    return dataframe.to_csv(filename)

## Get Tweets by trending topics on spesific country

In [4]:
df_tweets_trending = get_trending_topic_by_country('United States', 10)

Country: United States, WOEID: 23424977
#1: #Suicide
#2: Andrew Jackson
#3: #BreonnaTalyor
#4: #SuperStarBTS
#5: Christianity
#6: Mel Gibson
#7: Bill Simmons
#8: Katara
#9: Shaun King
#10: gowon


## Get Tweets by Keyword

We can pass trending topic or any keyword as a parameter to the function.

In [20]:
df_tweets = get_tweets_by_keyword(['covid', 'covid-19'], 100, 'en')

In [21]:
df_tweets

,username,screen_name,user_location,user_verified,followers_count,friends_count,listed_count,tweet_created_at,tweet_text,tweet_coordinates,tweet_place,tweet_favorite_count
0,grace⁷⋆ ѕтαу gσℓ∂ ⋆ 🐋,justjiminjamin,방탄소년단,False,1309,2469,11,2020-06-23 07:41:20,RT @erictile_: The reason why America isn’t re...,None,None,0
1,El Curly Headass🧢,ElCurlyHeadass,,False,44,286,0,2020-06-23 07:41:20,RT @erictile_: The reason why America isn’t re...,None,None,0
2,Future Tesla Owner,voktranes,,False,296,702,0,2020-06-23 07:41:20,RT @itskevo254: This is the first year I'm not...,None,None,0
3,मनीष कुशवाहा,mkushwaha266,"उत्तरप्रदेश, भारत",False,2902,2263,2,2020-06-23 07:41:20,RT @ipragyasingh1: #PriyankaJhoothiVadra DM ag...,None,None,0
4,Lifeisprecious 🎈 🇲🇻🇦🇺🇺🇸,LifeispreciousB,Australia,False,383,1161,7,2020-06-23 07:41:20,RT @tveitdal: The Covid-19 pandemic is threate...,None,None,0
...,...,...,...,...,...,...,...,...,...,...,...,...
95,AbdulGaniy,Ajakaiye_,As Allah leads.,False,334,2122,27,2020-06-23 07:41:06,@Gasbytweet And this isn’t trending. https://t...,None,None,0
96,🍯cami the bear🍯,stovetopbear,"Illinois, USA",False,465,905,0,2020-06-23 07:41:06,RT @BeatByBenally: Adding this late but the Na...,None,None,0
97,National Health Mission Manipur,health_manipur,,False,1014,273,0,2020-06-23 07:41:06,RT @airnews_imphal: #Manipur: Churachandpur Di...,None,None,0
98,ur fav,ihylaureennn,,False,161,149,0,2020-06-23 07:41:05,RT @erictile_: The reason why America isn’t re...,None,None,0


In [22]:
df_tweets['tweet_text'][1]

'RT @erictile_: The reason why America isn’t recovering from COVID-19 as quickly as other countries is because we are a country that values…'

## Remove Unwanted String from Tweets

In [23]:
df_tweets['tweet_text'] = remove_unwanted_string(df_tweets['tweet_text'])
print(df_tweets['tweet_text'])

0     @erictile_: The reason why America isn’t recov...
1     @erictile_: The reason why America isn’t recov...
2     @itskevo254: This is the first year I'm not go...
3     @ipragyasingh1: DM agra has issued u a notice ...
4     @tveitdal: The Covid-19 pandemic is threatenin...
                            ...                        
95     And this isn’t trending. https://t.co/vbzK1VUcSG
96    @BeatByBenally: Adding this late but the Navaj...
97    @airnews_imphal: Churachandpur District Hospit...
98    @erictile_: The reason why America isn’t recov...
99    @zeefa64: It only took 36 hours to get this ne...
Name: tweet_text, Length: 100, dtype: object


In [24]:
export_to_csv(df_tweets['tweet_text'], '../dataset/tweet_sample_100.csv')


Exported to CSV


## Tweet Sentiment with VaderSentiment library/module

Note: Can not support Indonesian language

In [ ]:
pos, neg, neu, comp = [], [], [], []
for text in df_tweets['tweet_text']:
    sentiment = sentiment_scores(text)
    pos.append(sentiment['pos'])
    neg.append(sentiment['neg'])
    neu.append(sentiment['neu'])
    comp.append(sentiment['compound'])
    
df_tweets_sentiment = pd.DataFrame()
# df_tweets_sentiment.columns = ['tweet_text', 'positive', 'negative', 'neutral', 'compound']
df_tweets_sentiment['tweet_text'] = df_tweets['tweet_text']
df_tweets_sentiment['positive'] = pos
df_tweets_sentiment['negative'] = neg
df_tweets_sentiment['neutral'] = neu
df_tweets_sentiment['compound'] = comp

In [ ]:
df_tweets_sentiment

,tweet_text,positive,negative,neutral,compound
0,@ScottGo18320060: COVID-19 Hoax - The Crime of...,0.101,0.176,0.723,-0.3400
1,@firstpost: L-G Anil Baijal ordered a 5-day ma...,0.085,0.000,0.915,0.0772
2,@Dhama_IPS: In our effort to contain Corona st...,0.000,0.000,1.000,0.0000
3,@brayson_lema: Being of service is by itself a...,0.123,0.083,0.795,0.2960
4,@f9hiMNCLFeo39HS: Containment of the new crisi...,0.000,0.177,0.823,-0.6249
5,"@hemantrajora_: Big Story : In AIIMS Delhi, 13...",0.153,0.000,0.847,0.5574
6,@Dhama_IPS: In our effort to contain Corona st...,0.000,0.000,1.000,0.0000
7,"@hemantrajora_: Big Story : In AIIMS Delhi, 13...",0.153,0.000,0.847,0.5574
8,Homemade healthy vegetable juice recipe. Read ...,0.213,0.000,0.787,0.4019
9,"@ARSHAD7649: If you are too close, you can bre...",0.000,0.000,1.000,0.0000


Detail information about the output visit https://github.com/cjhutto/vaderSentiment